In [1]:
# Vishakha Dixit
# 801265288
# HW-6 Problem-2 Solutions

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt  
import seaborn as sns 
from torchvision import transforms
from PIL import Image
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Part-1

from torchvision import datasets
data_path = '/content/drive/MyDrive/ML_HW/data'
cifar10 = datasets.CIFAR10(data_path, train = True, 
                           download = True, 
            transform =  transforms.Compose([transforms.ToTensor(), 
                            transforms.Normalize((0.4915, 0.4823, 0.4468), 
                                            (0.2470, 0.2435, 0.2616))]))
cifar10_val = datasets.CIFAR10(data_path, train = False, download = True, 
                    transform = transforms.Compose([transforms.ToTensor(), 
                            transforms.Normalize((0.4915, 0.4823, 0.4468), 
                                            (0.2470, 0.2435, 0.2616))]))

Files already downloaded and verified
Files already downloaded and verified


In [4]:
model = nn.Sequential(
            nn.Conv2d(3,16, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(16,8, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Linear(8 * 8 * 8, 32),
            nn.Tanh(),
            nn.Linear(32, 10))

In [5]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
    self.act1 = nn.Tanh()
    self.pool1 = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(16,8, kernel_size=3, padding=1)
    self.act2 = nn.Tanh()
    self.pool2 = nn.MaxPool2d(2)
    self.fc1 = nn.Linear(8*8*8,32)
    self.act3 = nn.Tanh()
    self.fc2 = nn.Linear(32,10)

  def forward(self, x):
    out = self.pool1(self.act1(self.conv1(x)))
    out = self.pool2(self.act2(self.conv2(out)))
    out = out.view(-1, 8*8*8)
    out = self.act3(self.fc1(out))
    out = self.fc2(out)
    return out

In [6]:
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(16,8, kernel_size=3, padding=1)
    self.fc1 = nn.Linear(8*8*8, 32)
    self.fc2 = nn.Linear(32, 10)
  
  def forward(self, x):
    out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
    out = F.max_pool2d(torch.tanh(self.conv2(out)),2)
    out = out.view(-1, 8*8*8)
    out = torch.tanh(self.fc1(out))
    out = self.fc2(out)
    return out

In [7]:
device = (torch.device('cuda:0') if torch.cuda.is_available() 
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda:0.


In [8]:
import datetime
def training_loop(n_epochs, optimizer, model, loss_fn, 
                  train_loader):
  for epoch in range(1, n_epochs +1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      outputs = model(imgs.to('cuda:0'))
      loss = loss_fn(outputs.to('cuda:0'), 
                     labels.to('cuda:0'))
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      loss_train += loss.item()
    if epoch == 1 or epoch % 1 == 0:
      print('{} Epoch {}, Training Loss {}'.format(datetime.datetime.now(), 
                                    epoch, loss_train / len(train_loader)))

In [9]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, 
                                           shuffle=True)

model = Net().to('cuda:0')
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs =300,
    optimizer= optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    
)

2022-12-04 01:27:59.121769 Epoch 1, Training Loss 2.050850319892854
2022-12-04 01:28:10.858518 Epoch 2, Training Loss 1.7718762310264666
2022-12-04 01:28:23.067879 Epoch 3, Training Loss 1.591920509210328
2022-12-04 01:28:34.991023 Epoch 4, Training Loss 1.4869809280271116
2022-12-04 01:28:48.265787 Epoch 5, Training Loss 1.4133399459712035
2022-12-04 01:28:59.784604 Epoch 6, Training Loss 1.3533931693152699
2022-12-04 01:29:10.918246 Epoch 7, Training Loss 1.300388334428563
2022-12-04 01:29:22.198832 Epoch 8, Training Loss 1.256114103376408
2022-12-04 01:29:33.497897 Epoch 9, Training Loss 1.2152485180541377
2022-12-04 01:29:44.776556 Epoch 10, Training Loss 1.1835584632880853
2022-12-04 01:29:55.809318 Epoch 11, Training Loss 1.1575147327407243
2022-12-04 01:30:07.031878 Epoch 12, Training Loss 1.1340423022847042
2022-12-04 01:30:18.469723 Epoch 13, Training Loss 1.1165185524984393
2022-12-04 01:30:29.956917 Epoch 14, Training Loss 1.0983067761601695
2022-12-04 01:30:41.370199 Epoch 

In [10]:
train_loader = torch.utils.data.DataLoader(cifar10, 
                    batch_size=64, shuffle=False)

val_loader = torch.utils.data.DataLoader(cifar10_val, 
                    batch_size=64, shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
      correct = 0
      total = 0
    
      with torch.no_grad():
        for imgs, labels in loader:
          outputs = model(imgs.to('cuda:0'))
          _, predicted = torch.max(outputs.to('cuda:0'), 
                                   dim=1)
          total += labels.shape[0]
          correct += int((predicted.to('cuda:0') == 
                          labels.to('cuda:0')).sum())
    
      print("Accuracy {}: {:.2f}".format(name, 
                                         correct/total))

In [11]:
validate(model, train_loader, val_loader)

Accuracy train: 0.82
Accuracy val: 0.62


In [12]:
# Part-2

In [13]:
model = nn.Sequential(
            nn.Conv2d(3,16, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(16,8, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(8,4, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Linear(4*4*4, 32),
            nn.Tanh(),
            nn.Linear(32, 10))

In [14]:
train_loader = torch.utils.data.DataLoader(cifar10, 
                    batch_size=64, shuffle=True)

model = Net().to('cuda:0')
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs =300,
    optimizer= optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    
)


2022-12-04 02:24:24.092688 Epoch 1, Training Loss 2.033293886715189
2022-12-04 02:24:34.869699 Epoch 2, Training Loss 1.7438221730844443
2022-12-04 02:24:45.781078 Epoch 3, Training Loss 1.5968979371478185
2022-12-04 02:24:56.733606 Epoch 4, Training Loss 1.5085458702138623
2022-12-04 02:25:07.628696 Epoch 5, Training Loss 1.4357301949540062
2022-12-04 02:25:18.421438 Epoch 6, Training Loss 1.367181647616579
2022-12-04 02:25:29.323309 Epoch 7, Training Loss 1.3099394624343004
2022-12-04 02:25:40.229547 Epoch 8, Training Loss 1.2627357881697243
2022-12-04 02:25:50.917148 Epoch 9, Training Loss 1.2238019455576796
2022-12-04 02:26:01.534904 Epoch 10, Training Loss 1.188430308685888
2022-12-04 02:26:12.245191 Epoch 11, Training Loss 1.1595776033066119
2022-12-04 02:26:23.074453 Epoch 12, Training Loss 1.1321494075495873
2022-12-04 02:26:33.680562 Epoch 13, Training Loss 1.1084413164107085
2022-12-04 02:26:45.459593 Epoch 14, Training Loss 1.088182297432819
2022-12-04 02:26:56.405973 Epoch 